<a href="https://colab.research.google.com/github/tackulus/229352/blob/main/Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import numpy as np
!wget -O dataset.csv \
    https://raw.githubusercontent.com/tackulus/229352/main/dataset/US-representatives-votes.csv

**1. Set the Numpy’s random seed.**



In [2]:
np.random.seed(0)

**2. Split the data into a training set and a test set.**

In [3]:
data = np.genfromtxt('dataset.csv', delimiter=',')

X, y = data[:,1:], data[:,0]

In [4]:
indexs, size = {}, len(data)
indexs['test']  = np.random.choice(range(size), size=int(0.25*size), replace=False)
indexs['train'] = [ n for n in range(size) if n not in indexs['test'] ]
indexs = {key: sorted(val) for key,val in indexs.items()}

X_train, X_test = X[indexs['train']], X[indexs['test']]
y_train, y_test = y[indexs['train']], y[indexs['test']]

**3. Create an array Count of shape (2,16,2), where Count[l,d,j] is the number of instances in the training set whose class is l and whose d-th feature is j.**

**In other words, Count[l,d,j] = #(d-th feature = j and Class = l).**

In [5]:
Count = np.zeros((2,16,2))
for i in indexs['train']:
    l = int(y[i])
    for d in range(16):
        j = int(X[i,d])
        Count[l,d,j] += 1

**4. Use Count to create another array Prob of shape (2,16,2), where Prob[l,d,j] is given by**

**Prob[l,d,j] = P (d-th feature = j | Class = l)**

In [6]:
Prob = np.zeros((2,16,2))

for l in range(2): 
    num = sum(Count[l,0,:])
    for d in range(16): 
        for j in range(2):
            Prob[l,d,j] = Count[l,d,j] / num

**5. To verify that you did the calculations correctly, check if the sum of each row in Prob[0] and Prob[1] are equal to 1.**

In [7]:
# Prob[0]
u = Prob[0,:,0] + Prob[0,:,1]
(u == 1).all()

True

In [8]:
# Prob[1]
u = Prob[1,:,0] + Prob[1,:,1]
(u == 1).all()

True

**6. Use Prob to classify each representative in the test set. Report the accuracy of your predictions (Note: I suggest computing the sum of log-probabilities instead of the product of probabilities to prevent numerical underflow).**

In [9]:
def predict(x):
    def f(x, j):
        return sum([ np.log(Prob[j,i,int(x[i])]) for i in range(16) ])

    a0, a1 = f(x,0), f(x,1)
    return int( a0 < a1 )

y_pred = np.apply_along_axis(predict, 1, X_test)

In [10]:
accuracy = sum( y_pred == y_test ) / len(y_pred)
accuracy

0.8796296296296297